In [1]:
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(argparse))

Warning message:
“package ‘ggplot2’ was built under R version 4.2.3”


## Three Beta Model Vizualization

In [ ]:
# define command line arguments
parser <- ArgumentParser(description = "Visualize linear modeling results")
# add arguments
parser$add_argument('--celltype', type='character', help='Cell type to visualize')

# parse arguments from command line
args <- parser$parse_args()

# define cell type
celltype <- args$celltype


In [2]:

lm_file <- file.path(paste0("./results/", celltype, "/lm_three_beta.tsv"))

lm_cp_fig <- file.path(paste0("./figures/", celltype, "/lm_three_beta.pdf"))

# if path does not exist, create it
if (!dir.exists(file.path(paste0("./figures/", celltype)))) {
    dir.create(file.path(paste0("./figures/", celltype)))
}
lm_df <- readr::read_tsv(lm_file, col_types = readr::cols(.default = "d", feature ="c", inducer1__inducer1_dose__inhibitor_inhibitor_dose = "c"))
head(lm_df)

feature,r2_score,Metadata_number_of_singlecells,threeb_Treatment,threeb_Treatment_Dose,threeb_Inhibitor_and_Dose,inducer1__inducer1_dose__inhibitor_inhibitor_dose
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Cytoplasm_AreaShape_Area,9.343787e-04,1.695962e-05,0.023208062,-0.023208062,0.023208062,media ctr__0__Media ctr_0.0
Cytoplasm_AreaShape_BoundingBoxArea,1.740451e-03,-3.367326e-05,0.023311221,-0.023311221,0.023311221,media ctr__0__Media ctr_0.0
Cytoplasm_AreaShape_BoundingBoxMaximum_X,1.077223e-04,-2.358903e-05,-0.008823020,0.008823020,-0.008823020,media ctr__0__Media ctr_0.0
Cytoplasm_AreaShape_BoundingBoxMaximum_Y,1.354607e-04,-2.231822e-05,0.003278083,-0.003278083,0.003278083,media ctr__0__Media ctr_0.0
Cytoplasm_AreaShape_BoundingBoxMinimum_X,1.515569e-04,-2.184418e-05,-0.010414957,0.010414957,-0.010414957,media ctr__0__Media ctr_0.0
Cytoplasm_AreaShape_BoundingBoxMinimum_Y,8.637752e-05,-2.176965e-05,0.001426285,-0.001426285,0.001426285,media ctr__0__Media ctr_0.0


In [3]:
unique(lm_df$inducer1__inducer1_dose__inhibitor_inhibitor_dose)

[1] "media ctr__0__Media ctr_0.0"               
 [2] "DMSO__0.100__DMSO_1.0"                     
 [3] "DMSO__0.100__Z-VAD-FMK_100.0"              
 [4] "DMSO__0.100__Z-VAD-FMK_30.0"               
 [5] "DMSO__0.100__DMSO_0.025"                   
 [6] "Thapsigargin__1.000__DMSO_0.025"           
 [7] "Thapsigargin__10.000__DMSO_0.025"          
 [8] "Topotecan__5.000__DMSO_0.025"              
 [9] "Topotecan__10.000__DMSO_0.025"             
[10] "Topotecan__20.000__DMSO_0.025"             
[11] "LPS__0.010__DMSO_0.025"                    
[12] "LPS__0.100__DMSO_0.025"                    
[13] "LPS__1.000__DMSO_0.025"                    
[14] "LPS__10.000__DMSO_0.025"                   
[15] "LPS__10.000__Disulfiram_0.1"               
[16] "LPS__10.000__Disulfiram_1.0"               
[17] "LPS__10.000__Disulfiram_2.5"               
[18] "LPS_Nigericin__100.000_1.0__DMSO_0.025"    
[19] "LPS_Nigericin__100.000_3.0__DMSO_0.025"    
[20] "LPS_Nigericin__100.000_10.0__DMSO_0.025"   
[21] "Disulfiram__0.100__DMSO_0.025"             
[22] "Disulfiram__1.000__DMSO_0.025"             
[23] "Disulfiram__2.500__DMSO_0.025"             
[24] "H2O2__100.000__DMSO_0.025"                 
[25] "LPS__10.000__Z-VAD-FMK_100.0"              
[26] "LPS__100.000__DMSO_0.025"                  
[27] "LPS_Nigericin__1.000_1.0__DMSO_0.025"      
[28] "LPS_Nigericin__1.000_3.0__DMSO_0.025"      
[29] "LPS_Nigericin__1.000_10.0__DMSO_0.025"     
[30] "LPS_Nigericin__1.000_10.0__Disulfiram_1.0" 
[31] "LPS_Nigericin__1.000_10.0__Z-VAD-FMK_100.0"
[32] "H2O2__100.000__Disulfiram_1.0"             
[33] "H2O2__100.000__Z-VAD-FMK_100.0"            
[34] "Flagellin__0.100__DMSO_0.025"              
[35] "Flagellin__1.000__DMSO_0.025"              
[36] "Flagellin__1.000__Disulfiram_1.0"

In [4]:

# Arrange by absolute value coefficient
# Split out components of feature name for visualization
lm_df <- lm_df %>%
    dplyr::arrange(desc(abs(threeb_Treatment_Dose))) %>%
    tidyr::separate(
        feature,
        into = c(
            "compartment",
            "feature_group",
            "measurement",
            "channel",
            "parameter1",
            "parameter2"
        ),
        sep = "_",
        remove = FALSE
    ) %>%
    dplyr::mutate(channel_cleaned = channel)



Warning message:
“Expected 6 pieces. Additional pieces discarded in 28080 rows [3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 16, 18, 19, 20, 23, 24, 25, 26, 27, ...].”
Warning message:
“Expected 6 pieces. Missing pieces filled with `NA` in 42012 rows [1, 2, 11, 14, 17, 21, 22, 40, 42, 45, 49, 52, 55, 56, 57, 64, 66, 67, 70, 76,
...].”


In [5]:
unique(lm_df$channel)

[1] "CorrMito"      "CorrPM"        "CorrDNA"       "CorrER"       
 [5] "CorrGasdermin" NA              "8"             "Adjacent"     
 [9] "0"             "3"             "Number"        "7"            
[13] "9"             "2"             "5"             "4"            
[17] "6"             "1"             "X"             "Y"            
[21] "Z"             "Count"

In [6]:
unique(lm_df$channel_cleaned)

[1] "CorrMito"      "CorrPM"        "CorrDNA"       "CorrER"       
 [5] "CorrGasdermin" NA              "8"             "Adjacent"     
 [9] "0"             "3"             "Number"        "7"            
[13] "9"             "2"             "5"             "4"            
[17] "6"             "1"             "X"             "Y"            
[21] "Z"             "Count"

In [7]:
# Clean channel for visualization
lm_df$channel_learned <- dplyr::recode(lm_df$channel_cleaned,
        "CorrDNA" = "nuclei",
        "CorrMito" = "Mito",
        "CorrER" = "ER",
        "CorrGasdermin" = "gasdermin",
        "CorrPM" = "PM",
        .default = "other",
        .missing="other"
    )

print(dim(lm_df))
head(lm_df)
unique(lm_df$channel_learned)
lm_df$abs_Metadata_number_of_singlecells <- abs(lm_df$Metadata_number_of_singlecells)
lm_df$abs_threeb_Treatment <- abs(lm_df$threeb_Treatment)
lm_df$abs_threeb_Treatment_Dose <- abs(lm_df$threeb_Treatment_Dose)
lm_df$abs_threeb_Inhibitor_and_Dose <- abs(lm_df$threeb_Inhibitor_and_Dose)

[1] 102492     15


feature,compartment,feature_group,measurement,channel,parameter1,parameter2,r2_score,Metadata_number_of_singlecells,threeb_Treatment,threeb_Treatment_Dose,threeb_Inhibitor_and_Dose,inducer1__inducer1_dose__inhibitor_inhibitor_dose,channel_cleaned,channel_learned
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
Nuclei_Intensity_MeanIntensityEdge_CorrMito,Nuclei,Intensity,MeanIntensityEdge,CorrMito,NA,NA,0.4552401,-0.0002177032,1.237168,1.237168,0,LPS_Nigericin__100.000_10.0__DMSO_0.025,CorrMito,Mito
Nuclei_Intensity_MaxIntensityEdge_CorrMito,Nuclei,Intensity,MaxIntensityEdge,CorrMito,NA,NA,0.4135021,-0.0002126844,1.168403,1.168403,0,LPS_Nigericin__100.000_10.0__DMSO_0.025,CorrMito,Mito
Cells_Texture_DifferenceEntropy_CorrMito_3_00_256,Cells,Texture,DifferenceEntropy,CorrMito,3,00,0.5400279,-0.0002267092,1.164350,1.164350,0,LPS_Nigericin__100.000_10.0__DMSO_0.025,CorrMito,Mito
Cells_Texture_DifferenceEntropy_CorrMito_3_02_256,Cells,Texture,DifferenceEntropy,CorrMito,3,02,0.5326017,-0.0002233205,1.145984,1.145984,0,LPS_Nigericin__100.000_10.0__DMSO_0.025,CorrMito,Mito
Cytoplasm_Texture_DifferenceEntropy_CorrMito_3_00_256,Cytoplasm,Texture,DifferenceEntropy,CorrMito,3,00,0.5165080,-0.0002200560,1.121624,1.121624,0,LPS_Nigericin__100.000_10.0__DMSO_0.025,CorrMito,Mito
Cytoplasm_Texture_DifferenceEntropy_CorrMito_3_02_256,Cytoplasm,Texture,DifferenceEntropy,CorrMito,3,02,0.5100435,-0.0002161120,1.105229,1.105229,0,LPS_Nigericin__100.000_10.0__DMSO_0.025,CorrMito,Mito


[1] "Mito"      "PM"        "nuclei"    "ER"        "gasdermin" "other"

In [8]:
head(lm_df)

feature,compartment,feature_group,measurement,channel,parameter1,parameter2,r2_score,Metadata_number_of_singlecells,threeb_Treatment,threeb_Treatment_Dose,threeb_Inhibitor_and_Dose,inducer1__inducer1_dose__inhibitor_inhibitor_dose,channel_cleaned,channel_learned,abs_Metadata_number_of_singlecells,abs_threeb_Treatment,abs_threeb_Treatment_Dose,abs_threeb_Inhibitor_and_Dose
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Nuclei_Intensity_MeanIntensityEdge_CorrMito,Nuclei,Intensity,MeanIntensityEdge,CorrMito,NA,NA,0.4552401,-0.0002177032,1.237168,1.237168,0,LPS_Nigericin__100.000_10.0__DMSO_0.025,CorrMito,Mito,0.0002177032,1.237168,1.237168,0
Nuclei_Intensity_MaxIntensityEdge_CorrMito,Nuclei,Intensity,MaxIntensityEdge,CorrMito,NA,NA,0.4135021,-0.0002126844,1.168403,1.168403,0,LPS_Nigericin__100.000_10.0__DMSO_0.025,CorrMito,Mito,0.0002126844,1.168403,1.168403,0
Cells_Texture_DifferenceEntropy_CorrMito_3_00_256,Cells,Texture,DifferenceEntropy,CorrMito,3,00,0.5400279,-0.0002267092,1.164350,1.164350,0,LPS_Nigericin__100.000_10.0__DMSO_0.025,CorrMito,Mito,0.0002267092,1.164350,1.164350,0
Cells_Texture_DifferenceEntropy_CorrMito_3_02_256,Cells,Texture,DifferenceEntropy,CorrMito,3,02,0.5326017,-0.0002233205,1.145984,1.145984,0,LPS_Nigericin__100.000_10.0__DMSO_0.025,CorrMito,Mito,0.0002233205,1.145984,1.145984,0
Cytoplasm_Texture_DifferenceEntropy_CorrMito_3_00_256,Cytoplasm,Texture,DifferenceEntropy,CorrMito,3,00,0.5165080,-0.0002200560,1.121624,1.121624,0,LPS_Nigericin__100.000_10.0__DMSO_0.025,CorrMito,Mito,0.0002200560,1.121624,1.121624,0
Cytoplasm_Texture_DifferenceEntropy_CorrMito_3_02_256,Cytoplasm,Texture,DifferenceEntropy,CorrMito,3,02,0.5100435,-0.0002161120,1.105229,1.105229,0,LPS_Nigericin__100.000_10.0__DMSO_0.025,CorrMito,Mito,0.0002161120,1.105229,1.105229,0


In [9]:
loop_list <- unique(lm_df$inducer1__inducer1_dose__inhibitor_inhibitor_dose)
x_list <- c('abs_threeb_Inhibitor_and_Dose','abs_threeb_Treatment','abs_threeb_Treatment_Dose')

In [10]:
pdf(file=lm_cp_fig )
for (i in 1:length(loop_list)){
    df <- lm_df[lm_df$inducer1__inducer1_dose__inhibitor_inhibitor_dose == loop_list[i],]
    for (j in 1:length(x_list)){
        lm_fig_gg <- (
            ggplot(df, aes(x = .data[[x_list[j]]], y = r2_score))
            + geom_point(aes(size = abs_Metadata_number_of_singlecells, color = channel_learned), alpha = 0.7)
            + theme_bw()
            + guides(
                color = guide_legend(title = "Channel\n(if applicable)", order = 1),
                size = guide_legend(title = "Cell count contributution")
            )
            + ylab("R2 score of LM feature")
            + xlab(paste0(x_list[j]," contribution (LM beta coefficient)"))
            + ggtitle(paste0("How CellProfiler features contribute\nto ",loop_list[i], "\ntreatments and cell density"))
        )
    plot(lm_fig_gg)    
    }
}
dev.off()

png 
  2